In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans

#### Data preparation

In [ ]:
#Desired format
'''
[{'diseaseId': ['C1252354', 'C1252355'], 'geneID': 1048},
 {'diseaseId': ['C1252486', 'C6142486'], 'geneID': 2011}]
 '''

In [3]:
gene_disease_csv = pd.read_csv("data/gene_disease_view.csv")
gene_disease_series = gene_disease_csv.groupby(["geneId"])["diseaseId"].apply(list)
gene_disease_dict = gene_disease_series.to_dict()

gene_to_list_of_diseases = []

for gene_id, disease_ids in gene_disease_dict.items():
    new_dict = {'diseaseId': disease_ids, 'geneID': gene_id}
    gene_to_list_of_diseases.append(new_dict)

#### Feature extraction

In [4]:
vec = DictVectorizer()
matrixGeneDisease = vec.fit_transform(gene_to_list_of_diseases)
matrixGeneDisease

<21671x30171 sparse matrix of type '<class 'numpy.float64'>'
	with 3282995 stored elements in Compressed Sparse Row format>

#### Clustering

In [8]:
cluster = KMeans()
result = cluster.fit(matrixGeneDisease)
result.labels_

array([0, 0, 0, ..., 4, 4, 4])